## Numpy - Applications in Finance

In [ ]:
import numpy as np

np.set_printoptions(suppress=True)

In [ ]:
datafile = 'http://people.bu.edu/kalathur/datasets/AAPL.csv'

In [ ]:
# Import closing price and volume from the file (columns 5 and 6)
# First column is column 0

c, v = np.loadtxt(datafile, delimiter=',', skiprows=1, usecols=(5,6), unpack=True)

c = np.round(c, 2)

In [ ]:
c[:5]

In [ ]:
v[:5]

In [ ]:
# number of values in the dataset

len(c)

**Volume Weighted Average Price (VWAP)**

In [ ]:
vwap = np.average(c, weights=v)
print("VWAP =", vwap)

In [ ]:
# Arithmetic mean

print("mean =", np.mean(c))


**Value Range**

In [ ]:
# Import daily high and low price from the file (columns 2 and 3)

h, l = np.loadtxt(datafile, delimiter=',', skiprows=1, usecols=(2,3), unpack=True)

h = np.round(h, 2)
l = np.round(l, 2)

In [ ]:
h[:5]

In [ ]:
l[:5]

In [ ]:
print("highest daily high =", np.max(h))
print("lowest  daily low =", np.min(l))

In [ ]:
# Spread of data

print("Spread high price", np.ptp(h))
print("Spread low price", np.ptp(l))

In [ ]:
print("Spread high price", np.max(h) - np.min(h))
print("Spread low price", np.max(l) - np.min(l))

**Statistics**

In [ ]:
print("median =", np.median(c))

In [ ]:
print("variance =", np.var(c))

In [ ]:
print("Standard deviation =", np.std(c))

**Simple Returns**
 - differences between consecutive values / value of the previous day

In [ ]:
returns = np.diff( c ) / c[ : -1]
returns[:5]

In [ ]:
np.max(returns), np.min(returns)

In [ ]:
print("Standard deviation =", np.std(returns))

**Log Returns**
 - log of all values and calculate differences between them
 - log(a) - log(b) = log(a/b)
 - measure rate of change
 - input should not have zeros or negative numbers

In [ ]:
logreturns = np.diff( np.log(c) )

In [ ]:
logreturns[:5]

In [ ]:
# Alternatively

logreturns = np.log(c[1:]/c[:-1])

logreturns[:5]

**Positive Returns**

In [ ]:
pos_ret_indices = np.where(returns > 0)
print("Indices with positive returns\n", pos_ret_indices)

In [ ]:
np.where(logreturns > 0)

**Volatility**
 - measures price variation
 - annualized volatility is equal to the standard deviation of the log returns as a ratio of its mean, divided by one over the square root of the number of business days in a year

In [ ]:
annual_volatility = np.std(logreturns)/np.mean(logreturns)
annual_volatility = annual_volatility / np.sqrt(1./252.)

print("Annual volatility", annual_volatility)


In [ ]:
print("Monthly volatility", annual_volatility * np.sqrt(1./12.))

**Dealing with Dates**

In [ ]:
from datetime import datetime
import calendar

Monday 0, Tuesday 1, Wednesday 2, Thursday 3, Friday 4, Saturday 5, Sunday 6

In [ ]:
list(calendar.day_name)

In [ ]:
def datestr2num(s):
   return datetime.strptime(s, "%Y-%m-%d").date().weekday()

In [ ]:
datestr2num('2019-9-30')

In [ ]:
datestr2num('2019-10-1')

In [ ]:
# Import Date and closing price from the file (columns 0 and 5)

dates, close = np.loadtxt(datafile, delimiter=',', 
                          skiprows=1, usecols=(0,5), 
                          converters={0: datestr2num},
                          encoding='utf-8',
                          unpack=True)

close = np.round(close, 2)

In [ ]:
close[:5]

In [ ]:
dates[:5]

In [ ]:
np.unique(dates)

In [ ]:
# averages based on day of week

averages = np.zeros(len(np.unique(dates)))

for i in range(5):
    indices = np.where(dates == i) 
    prices  = np.take(close, indices)
    avg = np.mean(prices)
    avg = np.round(avg)
    print("Day", i, "Average", avg)
    averages[i] = avg

In [ ]:
top = np.max(averages)
print("Highest average:", top)
print("Top day of the week index:", np.argmax(averages))
print("Top day of the week is", calendar.day_name[np.argmax(averages)])
print()

bottom = np.min(averages)
print("Lowest average:", bottom)
print("Bottom day of the week index:", np.argmin(averages))
print("Bottom day of the week is", calendar.day_name[np.argmin(averages)])

## Average True Range (ATR)
 - provides degree of price volatility
 - N-period smoothed moving average of the true range values
 - Recommended 14 period smoothing
 - Range of a day : $ (high-low) $
 - True Range TR = $ max[(high-low), abs(high-close_{prev}), abs(low-close_{prev})] $
 - $$ATR_{t} = \frac{ ATR_{t-1} * (N-1) + TR_{t} }{N}$$
 - First ATR is the arithmetic mean of the first N TR values
 
 

Example: https://school.stockcharts.com/doku.php?id=technical_indicators:average_true_range_atr

<img src="https://school.stockcharts.com/lib/exe/fetch.php?media=technical_indicators:average_true_range_atr:atr-2-qqqqsh.png"/>

In [ ]:
dates, open , high, low, close = np.loadtxt(datafile, delimiter=',',
                                     skiprows=1, usecols=(0,1,2,3,4), 
                                            converters={0: datestr2num},
                                            encoding='utf-8',
                                            unpack=True)

open  = np.round(open, 2)
high  = np.round(high, 2)
low   = np.round(low, 2)
close = np.round(close, 2)

In [ ]:
num_days = len(high)
num_days

In [ ]:
# ATR Window

N = 14

In [ ]:
previous_close = close[: -1]

In [ ]:
# TR except for first day

truerange = np.maximum(high[1:] - low[1:], 
                       np.abs(high[1:] - previous_close), 
                       np.abs(previous_close - low[1:]) )

# first day true range = (high[0] - low[0])
truerange = np.insert(truerange, 0, (high[0] - low[0]))

truerange


In [ ]:
len(truerange)

In [ ]:
atr = np.zeros(num_days - N + 1)
len(atr)

In [ ]:
atr[0] = np.mean(truerange[:N])
atr[0]

In [ ]:
for i in range(1, len(atr)):
   atr[i] = (N - 1) * atr[i - 1] + truerange[N + i - 1]
   atr[i] /= N

print("ATR", atr)

**Interpreting ATR and stock prices**

https://www.tradingview.com/wiki/Average_True_Range_(ATR)


In [ ]:
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc

In [ ]:
# Plot last 50 values

fig, ax = plt.subplots(2, figsize=(12,6))


t = np.arange(N - 1, num_days)
ax[0].plot(t[-50:], atr[-50:], '--', lw=2.0, label='ATR')
candlestick_ohlc(ax[1], zip(np.arange(len(high[-50:])),
                         open[-50:], high[-50:],
                         low[-50:], close[-50:]))

plt.xlabel('Days')
plt.show()

**Interpreting Moving Averages**

https://www.tradingview.com/wiki/Moving_Average

## Simple Moving Average (SMA)
 - For analyzing time-series data
 - Moving window of N periods
 - Mean of values inside the window
 - an unweighted moving average

In [ ]:
x = np.array([11,12,13,14,15,16,17,18])
x

In [ ]:
# 5-Day Moving Average
N = 5

In [ ]:
# First day of 5-day SMA
np.sum(x[0:N])/N

In [ ]:
# Second day of 5-day SMA
np.sum(x[1:N+1])/N

In [ ]:
# Third day of 5-day SMA
np.sum(x[2:N+2])/N

In [ ]:
# Fourth day of 5-day SMA
np.sum(x[3:N+3])/N

In [ ]:
# Using np.convolve

In [ ]:
N = 5
weights = np.ones(N)/N
print("Weights", weights)

In [ ]:
np.convolve(x, weights)

In [ ]:
np.convolve(x, weights)[N-1:-(N-1)]

In [ ]:
# Using the dataset

# Import Date and closing price from the file (columns 0 and 5)

dates, close = np.loadtxt(datafile, delimiter=',', 
                          skiprows=1, usecols=(0,5), 
                          converters={0: datestr2num},
                          encoding='utf-8',
                          unpack=True)

close = np.round(close, 2)

In [ ]:
len(close)

In [ ]:
# 20-day moving window

N = 20

In [ ]:
weights = np.ones(N)/N

In [ ]:
sma = np.convolve(c, weights)[N-1:-(N-1)]
len(sma)

In [ ]:
len(close[N-1:])

In [ ]:
t = np.arange(N - 1, len(close))

plt.plot(t, close[N-1:], lw=1.0, label="Data")

plt.plot(t, sma, '--', lw=2.0, label="SMA")

plt.title("20 Day Moving Average")
plt.xlabel("Days")
plt.ylabel("Price ($)")
plt.grid()
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(12,8))

candlestick_ohlc(ax, zip(np.arange(len(high[N-1:])),
                         open[N-1:], high[N-1:],
                         low[N-1:], close[N-1:]))

t1 = np.arange(0, len(close) - N + 1)

plt.plot(t1, close[N-1:], lw=2.0, label="Data")

plt.plot(t1, sma, '--', lw=2.0, label="SMA")


plt.xlabel('Days')

plt.grid()
plt.legend()
plt.show()


## Weighted Moving Average (WMA)

In [ ]:
x = np.array([5,6,7,8,9])
weights = np.array([5,4,3,2,1])
np.convolve(x, weights)/np.sum(weights)

In [ ]:
x = np.array([11,12,13,14,15,16,17,18])
x

In [ ]:
# 5-Day Moving Average
N = 5

In [ ]:
weights = np.arange(1, N+1)
weights

In [ ]:
# First day of 5-day WMA
np.sum(x[0:N] * weights)/sum(weights)

In [ ]:
# Second day of 5-day WMA
np.sum(x[1:N+1] * weights)/sum(weights)

In [ ]:
# Third day of 5-day WMA
np.sum(x[2:N+2] * weights)/sum(weights)

In [ ]:
# Fourth day of 5-day WMA
np.sum(x[3:N+3] * weights)/sum(weights)

In [ ]:
# Same as

(np.convolve(x, weights[::-1])[N-1:-(N-1)])/sum(weights)

In [ ]:
# Using the dataset

dates, close = np.loadtxt(datafile, delimiter=',', 
                          skiprows=1, usecols=(0,5), 
                          converters={0: datestr2num},
                          encoding='utf-8',
                          unpack=True)

close = np.round(close, 2)

In [ ]:
# 20-day moving window

N = 20

In [ ]:
weights = np.arange(1, N+1)
weights

In [ ]:
wma = (np.convolve(c, weights[::-1])[N-1:-(N-1)])/sum(weights)
wma[:5]

In [ ]:
fig, ax = plt.subplots(1, figsize=(12,8))

candlestick_ohlc(ax, zip(np.arange(len(high[N-1:])),
                         open[N-1:], high[N-1:],
                         low[N-1:], close[N-1:]))

t1 = np.arange(0, len(close) - N + 1)

plt.plot(t1, close[N-1:], lw=2.0, label="Data")

plt.plot(t1, sma, '--', lw=2.0, label="SMA")

plt.plot(t1, wma, '-.', lw=2.0, label="WMA")


plt.xlabel('Days')

plt.grid()
plt.legend()
plt.show()


## Exponential Moving Average (EMA)
 - For analyzing time-series data
 - Alternative to SMA
 - Moving window of N periods
 - Uses exponentially decreasing weights
 - Gives higher weights to recent prices
 

In [ ]:
x = np.array([11,12,13,14,15,16,17,18])
x

In [ ]:
N = 5
weights = np.exp(np.linspace(0, 1, N))
weights

In [ ]:
# Normalize weights
weights /= weights.sum()
print("Weights", weights)

In [ ]:
np.convolve(x, weights[::-1])[N-1:-(N-1)]

In [ ]:
# Using the dataset

# Import Date and closing price from the file (columns 0 and 5)

dates, close = np.loadtxt(datafile, delimiter=',', 
                          skiprows=1, usecols=(0,5), 
                          converters={0: datestr2num},
                          encoding='utf-8',
                          unpack=True)

close = np.round(close, 2)

In [ ]:
# 20-day moving window

N = 20

In [ ]:
weights = np.exp(np.linspace(0, 1, N))

# Normalize weights
weights /= weights.sum()

In [ ]:
ema = np.convolve(c, weights[::-1])[N-1:-(N-1)]
len(ema)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12,8))

candlestick_ohlc(ax, zip(np.arange(len(high[N-1:])),
                         open[N-1:], high[N-1:],
                         low[N-1:], close[N-1:]))

t1 = np.arange(0, len(close) - N + 1)

plt.plot(t1, close[N-1:], lw=2.0, label="Data")

plt.plot(t1, sma, '--', lw=2.0, label="SMA")

plt.plot(t1, wma, '-.', lw=2.0, label="WMA")

plt.plot(t1, ema, ':', lw=2.0, label="eMA")


plt.xlabel('Days')

plt.grid()
plt.legend()
plt.show()